<a href="https://colab.research.google.com/github/FedeHorus/02-formularios-html/blob/main/QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl faster-whisper requests ipywidgets

import os
import re
import logging
import tempfile
import pandas as pd
import requests
from datetime import datetime
from faster_whisper import WhisperModel
from google.colab import files, drive
import ipywidgets as widgets
from IPython.display import display, HTML
import torch

# Configurar logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Montar Google Drive
drive.mount('/content/drive')

# Configuración centralizada (de config.py)
WHISPER_CONFIG = {
    "model_size": "medium",  # Modelo equilibrado para precisión y velocidad
    "device": "cuda",  # Usamos GPU (CUDA) para acelerar la transcripción
    "beam_size": 5,  # Aumenta precisión en la decodificación
    "vad_filter": True,  # Filtra segmentos sin voz
    "temperature": 0,  # Transcripciones deterministas
    "language": None,  # Detección automática de idioma
    "no_speech_threshold": 0.6  # Umbral razonable para filtrar ruido
}

DOWNLOAD_CONFIG = {
    "timeout": 60,
    "chunk_size": 8192,
    "max_file_size": 100 * 1024 * 1024,
    "allowed_extensions": ['.mp3', '.wav', '.m4a', '.mp4'],
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

VERTICAL_RULES = {
    "ACA": {
        "keywords": [
            r"health insurance", r"affordable care act", r"obamacare", r"medical coverage",
            r"health plan", r"insurance marketplace", r"premium tax credit", r"essential health benefits"
        ],
        "flags": [
            r"6500 gift card", r"free gift card", r"guaranteed approval", r"no medical exam",
            r"pre-existing conditions covered"
        ],
        "priority": 1
    },
    "Medicare": {
        "keywords": [
            r"medicare", r"medicare advantage", r"medicare supplement", r"medigap",
            r"part a", r"part b", r"part c", r"part d", r"senior benefits", r"medicare enrollment"
        ],
        "flags": [
            r"no premium", r"zero cost", r"extra benefits", r"dental and vision included",
            r"prescription drug coverage"
        ],
        "priority": 2
    },
    "Debt": {
        "keywords": [
            r"debt relief", r"debt consolidation", r"debt settlement", r"credit card debt",
            r"personal loan", r"financial hardship", r"alivio de la deuda", r"alivio para sus deudas", r"debt"
        ],
        "flags": [
            r"government program", r"cancel debt", r"eliminate debt", r"forgiveness program",
            r"reduce debt by \d+%", r"stop paying creditors", r"programa del gobierno", r"estan dando dinero", r"pay back"
        ],
        "priority": 3
    },
    "Solar": {
        "keywords": [
            r"solar panels", r"solar energy", r"renewable energy", r"solar installation",
            r"solar system", r"clean energy", r"solar", r"electricity"
        ],
        "flags": [
            r"no cost solar", r"free solar panels", r"free solar system", r"government incentive",
            r"tax credit", r"zero down payment", r"eliminate electric bill"
        ],
        "priority": 4
    },
    "Roofing": {
        "keywords": [
            r"roof repair", r"roof replacement", r"roofing services", r"home improvement",
            r"storm damage"
        ],
        "flags": [
            r"free roofing", r"insurance will pay", r"no out of pocket", r"government program", r"free roof"
        ],
        "priority": 5
    },
    "Auto Insurance": {
        "keywords": [
            r"car insurance", r"auto insurance", r"vehicle insurance", r"auto coverage",
            r"liability insurance", r"comprehensive coverage"
        ],
        "flags": [
            r"lower your rates", r"save \$\d+ per month", r"compare quotes", r"switch and save"
        ],
        "priority": 6
    },
    "Home Security": {
        "keywords": [
            r"home security", r"security system", r"alarm system", r"home monitoring",
            r"security cameras"
        ],
        "flags": [
            r"free installation", r"free equipment", r"government discount", r"senior discount"
        ],
        "priority": 7
    }
}

QUALITY_PATTERNS = {
    "positive_indicators": [
        r"thank you", r"gracias", r"appreciate", r"interested", r"tell me more",
        r"cuando puedo", r"how much", r"cuanto cuesta", r"appointment set" , r"you are done"
    ],
    "negative_indicators": [
        r"not interested", r"no me interesa", r"remove.*list", r"don't call",
        r"no llamen", r"hang up", r"disconnect", r"stop calling", r"take me off"
    ],
    "compliance_issues": [
        r"guaranteed", r"risk free", r"no obligation", r"limited time",
        r"act now", r"call within", r"expires today",r"free roof", r"government program"
    ]
}

EXCEL_COLUMN_MAPPING = {
    'recording_link': [
        'Recording Link', 'Recording URL', 'recording_url', 'URL', 'url',
        'Audio URL', 'Link', 'recording_link', 'audio_link', 'call_recording'
    ],
    'caller_id': [
        'Caller ID', 'CallID', 'caller_id', 'ID', 'Phone', 'Caller',
        'phone_number', 'PhoneNumber', 'lead_phone', 'customer_phone'
    ],
    'publisher': ['Publisher', 'publisher', 'Pub', 'Source', 'Traffic Source', 'Partner'],
    'buyer': ['Buyer', 'buyer', 'Client', 'Customer', 'Account'],
    'date': ['Date', 'date', 'Call Date', 'Timestamp', 'Created Date', 'call_date'],
    'time': ['Time', 'time', 'Call Time', 'Duration', 'Call Duration', 'Length'],
    'campaign': ['Campaign', 'campaign', 'Campaign Name', 'Campaign ID'],
    'lead_id': ['Lead ID', 'lead_id', 'LeadID', 'Lead', 'Lead Number'],
    'vertical': ['Vertical', 'vertical', 'Industry', 'Category', 'Product'],
    'status': ['Status', 'status', 'Call Status', 'Disposition']
}

# Funciones de analyzer.py
model = WhisperModel(
    WHISPER_CONFIG["model_size"],
    device=WHISPER_CONFIG["device"],
    compute_type="float16" if WHISPER_CONFIG["device"] == "cuda" else "int8"
)

def transcribe(audio_path):
    start = time.time()
    try:
        segments, info = model.transcribe(
            audio_path,
            beam_size=WHISPER_CONFIG["beam_size"],
            vad_filter=WHISPER_CONFIG["vad_filter"],
            temperature=WHISPER_CONFIG["temperature"],
            language=WHISPER_CONFIG["language"],
            no_speech_threshold=WHISPER_CONFIG["no_speech_threshold"]
        )
        text = " ".join([s.text.strip() for s in segments]).lower()
        language = info.language
        confidence = getattr(info, 'language_probability', 0.0)
        logger.info(f"⏱️ Transcripción de {audio_path} tomó {time.time() - start:.2f} segundos")
        if WHISPER_CONFIG["device"] == "cuda":
            torch.cuda.empty_cache()  # Liberar memoria GPU después de la transcripción
        return text, language, confidence
    except Exception as e:
        logger.error(f"Error en transcripción: {e}")
        if WHISPER_CONFIG["device"] == "cuda":
            torch.cuda.empty_cache()  # Liberar memoria GPU en caso de error
        return "", "error", 0.0

def detect_vertical_and_flags(text):
    detected_verticals = {}
    detected_flags = {}
    for vertical, config in VERTICAL_RULES.items():
        # Detectar vertical usando keywords
        keyword_matches = []
        for pattern in config["keywords"]:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                keyword_matches.extend([pattern.replace(r'\b', '').replace(r'\.', '.') for _ in matches])
        if keyword_matches:
            detected_verticals[vertical] = keyword_matches

        # Detectar flags usando flags
        flag_matches = []
        for pattern in config["flags"]:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                flag_matches.extend([pattern.replace(r'\b', '').replace(r'\.', '.') for _ in matches])
        if flag_matches:
            detected_flags[vertical] = flag_matches

    if detected_verticals:
        # Seleccionar la vertical principal basada en la cantidad de keywords
        primary_vertical = max(detected_verticals.keys(), key=lambda k: len(detected_verticals[k]))
        # Obtener los flags asociados solo a la vertical principal, si los hay
        primary_flags = detected_flags.get(primary_vertical, [])
        return primary_vertical, primary_flags, detected_verticals, detected_flags
    return "No asignada", [], {}, {}

def generate_summary(text, flags, vertical):
    summary_parts = []
    word_count = len(text.split())
    summary_parts.append(f"Transcripción de {word_count} palabras")
    if vertical != "No asignada":
        summary_parts.append(f"Vertical detectada: {vertical}")
    if flags:
        summary_parts.append(f"Flags encontrados: {len(flags)} ({', '.join(flags[:3])}{'...' if len(flags) > 3 else ''})")
    else:
        summary_parts.append("Sin flags detectados")
    if "thank you" in text or "gracias" in text:
        summary_parts.append("Llamada cortés")
    if len(text) < 50:
        summary_parts.append("⚠️ Transcripción muy corta")
    return " | ".join(summary_parts)

def validate_call_quality(text, flags):
    if not text.strip():
        return "No cumple", ["Sin transcripción"]
    issues = []
    if len(text.split()) < 10:
        issues.append("Transcripción muy corta")

    for pattern in QUALITY_PATTERNS["negative_indicators"]:
        if re.search(pattern, text, re.IGNORECASE):
            issues.append(f"Contenido problemático: {pattern}")
    return "No cumple" if issues else "Cumple", issues

def extract_caller_id_from_filename(filename):
    patterns = [r'\b\d{10,15}\b', r'caller[_-]?(\d+)', r'id[_-]?(\d+)', r'phone[_-]?(\d+)']
    for pattern in patterns:
        match = re.search(pattern, filename, re.IGNORECASE)
        if match:
            return match.group(1) if len(match.groups()) > 0 else match.group()
    return "N/A"
def generate_transcript_summary(transcript, flags, vertical):
    """Genera un resumen breve de la transcripción basado en patrones y contenido."""
    if not transcript.strip():
        return "Sin transcripción disponible"

    summary_parts = []

    # Detectar interés o desinterés
    for pattern in QUALITY_PATTERNS["negative_indicators"]:
        if re.search(pattern, transcript, re.IGNORECASE):
            summary_parts.append("no está interesado")
            break
    else:
        for pattern in QUALITY_PATTERNS["positive_indicators"]:
            if re.search(pattern, transcript, re.IGNORECASE):
                summary_parts.append("muestra interés")
                break

    # Añadir tema principal basado en flags o vertical
    if flags:
        # Seleccionar el flag más relevante (el primero detectado)
        main_flag = flags[0]
        if "government program" in main_flag.lower():
            summary_parts.append("preguntó por programa del gobierno")
        elif "free" in main_flag.lower():
            summary_parts.append(f"preguntó por {main_flag.lower()}")
        else:
            summary_parts.append(f"mencionó {main_flag.lower()}")
    elif vertical != "No asignada":
        # Si no hay flags, usar la vertical
        summary_parts.append(f"habló sobre {vertical.lower()}")

    # Si no hay información específica, usar un resumen genérico
    if not summary_parts:
        summary_parts.append("conversación genérica")

    # Combinar partes en una frase
    summary = ", ".join(summary_parts)

    # Asegurar que el resumen sea breve
    words = summary.split()
    if len(words) > 15:
        summary = " ".join(words[:15]) + "..."

    # Capitalizar primera letra
    return summary[0].upper() + summary[1:] if summary else "Sin resumen"

def analyze_audio(audio_path, recording_url=None, publisher=None, buyer=None, caller_id=None, additional_info=None):
    try:
        transcript, language, confidence = transcribe(audio_path)
        vertical, flags, all_verticals, detected_flags = detect_vertical_and_flags(transcript)
        summary = generate_summary(transcript, flags, vertical)
        compliance_status, quality_issues = validate_call_quality(transcript, flags)
        transcript_summary = generate_transcript_summary(transcript, flags, vertical)
        if not caller_id:
            caller_id = extract_caller_id_from_filename(os.path.basename(audio_path))
        result = {
            "Caller ID": caller_id,
            "Publisher": publisher or "N/A",
            "Buyer": buyer or "N/A",
            "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Recording Link": recording_url or audio_path,
            "Vertical": vertical,
            "Flags Found": ", ".join(flags) if flags else "Ninguna",
            "Flags Count": len(flags),
            "Summary": summary,
            "Compliance": compliance_status,
            "Language": language,
            "Language Confidence": f"{confidence:.2f}" if confidence else "N/A",
            "Transcript Length": len(transcript.split()),
            "Full Transcript": transcript,
            "Quality Issues": ", ".join(quality_issues) if quality_issues else "None",
            "Additional Info": str(additional_info) if additional_info else "N/A",
            "Transcript Summary": transcript_summary
        }
        return result
    except Exception as e:
        logger.error(f"Error analyzing audio {audio_path}: {e}")
        return {
            "Caller ID": caller_id or "Error",
            "Publisher": publisher or "N/A",
            "Buyer": buyer or "N/A",
            "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Recording Link": recording_url or audio_path,
            "Vertical": "Error",
            "Flags Found": "Error",
            "Flags Count": 0,
            "Summary": f"Error en análisis: {str(e)}",
            "Compliance": "Error",
            "Language": "Error",
            "Language Confidence": "N/A",
            "Transcript Length": 0,
            "Full Transcript": f"Error: {str(e)}",
            "Quality Issues": str(e),
            "Additional Info": "Error en procesamiento",
            "Transcript Summary": "Error en transcripción"
        }

# Funciones de app.py adaptadas
def download_audio_from_url(url, timeout=DOWNLOAD_CONFIG["timeout"]):
    try:
        logger.info(f"Descargando audio desde: {url}")
        headers = {'User-Agent': DOWNLOAD_CONFIG["user_agent"]}
        response = requests.get(url, timeout=timeout, headers=headers, stream=True)
        response.raise_for_status()
        content_type = response.headers.get('content-type', '').lower()
        suffix = '.wav' if 'audio/wav' in content_type else '.m4a' if 'audio/mp4' in content_type or 'audio/m4a' in content_type else '.mp3'
        with tempfile.NamedTemporaryFile(delete=False, suffix=suffix, dir='/content') as temp_file:
            for chunk in response.iter_content(chunk_size=DOWNLOAD_CONFIG["chunk_size"]):
                if chunk:
                    temp_file.write(chunk)
            temp_path = temp_file.name
        if os.path.getsize(temp_path) == 0:
            os.remove(temp_path)
            logger.error("Archivo descargado está vacío")
            return None
        logger.info(f"Audio descargado: {temp_path}")
        return temp_path
    except Exception as e:
        logger.error(f"Error descargando {url}: {e}")
        return None

def read_excel_with_enhanced_mapping(excel_file):
    try:
        logger.info(f"Leyendo archivo Excel: {excel_file}")
        df = pd.read_excel(excel_file)
        df.columns = df.columns.str.strip()
        found_columns = {}
        for key, possible_names in EXCEL_COLUMN_MAPPING.items():
            for col_name in possible_names:
                if col_name in df.columns:
                    found_columns[key] = col_name
                    break
        if 'recording_link' not in found_columns:
            raise ValueError(f"No se encontró columna de URLs. Columnas disponibles: {list(df.columns)}")
        url_column = found_columns['recording_link']
        df_valid = df[df[url_column].notna() & (df[url_column] != '') & (df[url_column].str.strip() != '')]
        if len(df_valid) == 0:
            raise ValueError("No se encontraron URLs válidas en el archivo")
        processing_info = []
        for idx, row in df_valid.iterrows():
            info = {
                'url': str(row[url_column]).strip(),
                'row_index': idx,
                'data': {key: str(row[col]).strip() for key, col in found_columns.items() if pd.notna(row[col])}
            }
            additional_info = {col: str(row[col]).strip() for col in df.columns if col not in found_columns.values() and pd.notna(row[col])}
            if additional_info:
                info['data']['additional'] = additional_info
            processing_info.append(info)
        logger.info(f"Preparadas {len(processing_info)} filas para procesar")
        return df, processing_info, found_columns
    except Exception as e:
        logger.error(f"Error leyendo Excel: {e}")
        raise

def process_excel_enhanced(excel_file, progress_callback=None):
    try:
        if progress_callback:
            progress_callback(0.1, "Leyendo archivo Excel...")
        df, processing_info, found_columns = read_excel_with_enhanced_mapping(excel_file)
        results = []
        total_items = len(processing_info)
        logger.info(f"Iniciando procesamiento de {total_items} elementos del Excel")
        for i, item in enumerate(processing_info):
            if progress_callback:
                progress_callback((i + 1) / total_items, f"Procesando fila {item['row_index'] + 1}/{total_items}")
            try:
                url = item['url']
                row_data = item['data']
                audio_path = download_audio_from_url(url)
                if audio_path:
                    try:
                        result = analyze_audio(
                            audio_path=audio_path,
                            recording_url=url,
                            publisher=row_data.get('publisher'),
                            buyer=row_data.get('buyer'),
                            caller_id=row_data.get('caller_id'),
                            additional_info=row_data.get('additional', {})
                        )
                        result['Excel Row'] = item['row_index'] + 1
                        result['Original Data'] = str(row_data)
                        results.append(result)
                        logger.info(f"✅ Procesado exitosamente fila {item['row_index'] + 1}")
                    finally:
                        try:
                            os.remove(audio_path)
                        except Exception as e:
                            logger.warning(f"Error eliminando archivo temporal: {e}")
                else:
                    error_result = {
                        'Excel Row': item['row_index'] + 1,
                        'Caller ID': row_data.get('caller_id', 'N/A'),
                        'Recording Link': url,
                        'Vertical': 'Error',
                        'Flags Found': 'Error de descarga',
                        'Flags Count': 0,
                        'Summary': f'No se pudo descargar audio desde: {url}',
                        'Compliance': 'No cumple',
                        'Language': 'Error',
                        'Full Transcript': 'Error: No se pudo descargar el audio',
                        'Original Data': str(row_data)
                    }
                    results.append(error_result)
                    logger.error(f"❌ Error descargando fila {item['row_index'] + 1}: {url}")
            except Exception as e:
                logger.error(f"Error procesando fila {item['row_index'] + 1}: {e}")
                error_result = {
                    'Excel Row': item['row_index'] + 1,
                    'Caller ID': row_data.get('caller_id', 'Error'),
                    'Recording Link': url,
                    'Vertical': 'Error',
                    'Flags Found': 'Error de procesamiento',
                    'Summary': f'Error en procesamiento: {str(e)}',
                    'Compliance': 'Error',
                    'Original Data': str(row_data)
                }
                results.append(error_result)
        results_df = pd.DataFrame(results)
        if 'Excel Row' in results_df.columns:
            results_df = results_df.sort_values('Excel Row')
        logger.info(f"Procesamiento Excel completado: {len(results)} resultados")
        return results_df
    except Exception as e:
        logger.error(f"Error en procesamiento Excel: {e}")
        return pd.DataFrame([{
            'Error': f'Error procesando archivo Excel: {str(e)}',
            'Caller ID': 'Error',
            'Summary': f'No se pudo procesar el archivo: {str(e)}'
        }])

def process_mixed_inputs(files, urls_text, excel_file, progress_callback=None):
    import time  # Aseguramos importar time
    start_time = time.time()  # Registrar inicio
    try:
        all_results = []
        if excel_file:
            if progress_callback:
                progress_callback(0, "Procesando archivo Excel...")
            excel_results = process_excel_enhanced(excel_file, progress_callback)
            all_results.append(('Excel', excel_results))
        if files:
            if progress_callback:
                progress_callback(0.7, "Procesando archivos individuales...")
            file_results = []
            for i, file_obj in enumerate(files):
                try:
                    result = analyze_audio(file_obj.name)
                    result['Source'] = 'Archivo local'
                    result['File Name'] = os.path.basename(file_obj.name)
                    file_results.append(result)
                except Exception as e:
                    logger.error(f"Error procesando archivo {file_obj.name}: {e}")
            if file_results:
                files_df = pd.DataFrame(file_results)
                all_results.append(('Archivos', files_df))
        if urls_text and urls_text.strip():
            if progress_callback:
                progress_callback(0.9, "Procesando URLs manuales...")
            urls = [url.strip() for url in urls_text.split(",") if url.strip()]
            url_results = []
            for i, url in enumerate(urls):
                try:
                    audio_path = download_audio_from_url(url)
                    if audio_path:
                        try:
                            result = analyze_audio(audio_path, recording_url=url)
                            result['Source'] = 'URL manual'
                            result['URL Index'] = i + 1
                            url_results.append(result)
                        finally:
                            os.remove(audio_path)
                except Exception as e:
                    logger.error(f"Error procesando URL {url}: {e}")
            if url_results:
                urls_df = pd.DataFrame(url_results)
                all_results.append(('URLs', urls_df))
        if all_results:
            combined_df = pd.concat([df for _, df in all_results], ignore_index=True)
            total_time = time.time() - start_time  # Calcular tiempo total
            if progress_callback:
                progress_callback(1.0, f"Completado: {len(combined_df)} audios procesados")
            return combined_df, total_time  # Retornar DataFrame y tiempo
        return pd.DataFrame([{
            'Error': 'No se proporcionaron archivos para procesar',
            'Caller ID': 'N/A',
            'Summary': 'Proporciona archivos de audio, URLs o un archivo Excel'
        }]), 0
    except Exception as e:
        logger.error(f"Error en procesamiento general: {e}")
        return pd.DataFrame([{
            'Error': f'Error general: {str(e)}',
            'Caller ID': 'Error',
            'Summary': 'Error en el procesamiento general'
        }]), 0

def export_enhanced_excel(df):
    try:
        output_path = '/content/drive/My Drive/call_analysis_results.xlsx'
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Análisis Completo', index=False)
            if len(df) > 0:
                summary_data = {
                    'Métrica': [
                        'Total Audios', 'Cumple Calidad', 'No Cumple', 'Con Flags', 'Sin Flags',
                        'Idioma Español', 'Idioma Inglés'
                    ],
                    'Valor': [
                        len(df),
                        len(df[df.get('Compliance', '') == 'Cumple']),
                        len(df[df.get('Compliance', '') == 'No cumple']),
                        len(df[df.get('Flags Count', 0) > 0]),
                        len(df[df.get('Flags Count', 0) == 0]),
                        len(df[df.get('Language', '') == 'es']),
                        len(df[df.get('Language', '') == 'en'])
                    ]
                }
                summary_df = pd.DataFrame(summary_data)
                summary_df.to_excel(writer, sheet_name='Resumen', index=False)
        logger.info(f"Excel exportado a {output_path}")
        return output_path
    except Exception as e:
        logger.error(f"Error exportando Excel: {e}")
        return None

def generate_statistics_markdown(df, total_time):
    try:
        total = len(df)
        if total == 0:
            return "Sin datos para mostrar estadísticas"
        cumple = len(df[df.get('Compliance', '') == 'Cumple']) if 'Compliance' in df.columns else 0
        no_cumple = len(df[df.get('Compliance', '') == 'No cumple']) if 'Compliance' in df.columns else 0
        con_flags = len(df[df.get('Flags Count', 0) > 0]) if 'Flags Count' in df.columns else 0
        languages = df.get('Language', pd.Series()).value_counts().to_dict() if 'Language' in df.columns else {}
        verticals = df.get('Vertical', pd.Series()).value_counts().to_dict() if 'Vertical' in df.columns else {}

        # Formatear el tiempo total
        minutes = int(total_time // 60)
        seconds = int(total_time % 60)
        time_str = f"{minutes} min {seconds} seg" if minutes > 0 else f"{seconds} seg"

        stats_md = f"""
## 📊 Estadísticas del Análisis

### 📈 Resumen General
- **Total de audios procesados:** {total}
- **Llamadas que cumplen:** {cumple} ({cumple/total*100:.1f}%)
- **Llamadas que no cumplen:** {no_cumple} ({no_cumple/total*100:.1f}%)
- **Llamadas con flags:** {con_flags} ({con_flags/total*100:.1f}%)
- **Tiempo total de análisis:** {time_str}

### 🌐 Distribución por Idioma
"""
        for lang, count in languages.items():
            percentage = count/total*100
            stats_md += f"- **{lang.upper()}:** {count} audios ({percentage:.1f}%)\n"
        stats_md += "\n### 🎯 Distribución por Vertical\n"
        for vertical, count in verticals.items():
            percentage = count/total*100
            stats_md += f"- **{vertical}:** {count} audios ({percentage:.1f}%)\n"
        if 'Flags Found' in df.columns:
            all_flags = []
            for flags_str in df['Flags Found'].dropna():
                if flags_str != 'Ninguna' and flags_str != 'Error':
                    all_flags.extend([f.strip() for f in str(flags_str).split(',')])
            if all_flags:
                flag_counts = pd.Series(all_flags).value_counts().head(5)
                stats_md += "\n### 🚩 Top 5 Flags Más Comunes\n"
                for flag, count in flag_counts.items():
                    stats_md += f"- **{flag}:** {count} veces\n"
        return stats_md
    except Exception as e:
        return f"Error generando estadísticas: {str(e)}"

# Interfaz con ipywidgets
def create_ui():
    excel_input = widgets.FileUpload(accept='.xlsx,.xls', multiple=False, description='Subir Excel')
    audio_input = widgets.FileUpload(accept='.mp3,.wav,.m4a', multiple=True, description='Subir Audios')
    url_input = widgets.Textarea(value='', placeholder='https://ejemplo.com/audio1.mp3, https://ejemplo.com/audio2.mp3', description='URLs:', layout={'width': '600px', 'height': '100px'})
    process_button = widgets.Button(description='Procesar Todo', button_style='primary')
    status_output = widgets.Output()
    results_output = widgets.Output()
    stats_output = widgets.Output()

    def process_all_inputs(b):
        with status_output:
            status_output.clear_output()
            results_output.clear_output()
            stats_output.clear_output()
            print("⏳ Iniciando procesamiento...")

        files = []
        for filename, file_info in audio_input.value.items():
            with open(f'/content/{filename}', 'wb') as f:
                f.write(file_info['content'])
            files.append(type('obj', (object,), {'name': f'/content/{filename}'}))

        excel_file = None
        if excel_input.value:
            filename = list(excel_input.value.keys())[0]
            with open(f'/content/{filename}', 'wb') as f:
                f.write(excel_input.value[filename]['content'])
            excel_file = f'/content/{filename}'

        def progress_callback(progress, message):
            with status_output:
                status_output.clear_output()
                print(f"{message} ({progress*100:.1f}%)")

        try:
            df, total_time = process_mixed_inputs(files, url_input.value, excel_file, progress_callback)
            if len(df) == 0:
                with status_output:
                    status_output.clear_output()
                    print("❌ No se encontraron datos para procesar")
                # Limpiar widgets y archivos temporales
                excel_input._counter = 0  # Resetear FileUpload
                audio_input._counter = 0  # Resetear FileUpload
                url_input.value = ''
                # Limpiar archivos temporales
                import glob
                for temp_file in glob.glob('/content/*.xlsx') + glob.glob('/content/*.mp3') + glob.glob('/content/*.wav') + glob.glob('/content/*.m4a'):
                    try:
                        os.remove(temp_file)
                    except Exception as e:
                        logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")
                return

            preview_cols = ["Caller ID", "Vertical", "Flags Found", "Transcript Summary", "Compliance"]
            preview_df = df[preview_cols].head(50) if all(col in df.columns for col in preview_cols) else df.head(50)
            stats_md = generate_statistics_markdown(df, total_time)
            excel_path = export_enhanced_excel(df)

            with results_output:
                results_output.clear_output()
                display(HTML("<h3>Resultados</h3>"))
                display(preview_df)

            with stats_output:
                stats_output.clear_output()
                display(HTML("<h3>Estadísticas</h3>"))
                display(HTML(stats_md.replace('\n', '<br>')))

            with status_output:
                status_output.clear_output()
                if excel_path:
                    print(f"✅ Procesamiento completado: {len(df)} audios analizados")
                    display(HTML(f'<a href="{excel_path}" download>Descargar Reporte Excel</a>'))
                else:
                    print(f"⚠️ Procesados {len(df)} audios, pero error exportando Excel")

            # Limpiar widgets y archivos temporales después de procesar
            excel_input._counter = 0  # Resetear FileUpload
            audio_input._counter = 0  # Resetear FileUpload
            url_input.value = ''
            # Limpiar archivos temporales
            import glob
            for temp_file in glob.glob('/content/*.xlsx') + glob.glob('/content/*.mp3') + glob.glob('/content/*.wav') + glob.glob('/content/*.m4a'):
                try:
                    os.remove(temp_file)
                except Exception as e:
                    logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")

        except Exception as e:
            with status_output:
                status_output.clear_output()
                print(f"❌ Error: {str(e)}")
            # Limpiar widgets y archivos temporales en caso de error
            excel_input._counter = 0  # Resetear FileUpload
            audio_input._counter = 0  # Resetear FileUpload
            url_input.value = ''
            # Limpiar archivos temporales
            import glob
            for temp_file in glob.glob('/content/*.xlsx') + glob.glob('/content/*.mp3') + glob.glob('/content/*.wav') + glob.glob('/content/*.m4a'):
                try:
                    os.remove(temp_file)
                except Exception as e:
                    logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")

    process_button.on_click(process_all_inputs)
    display(HTML("<h2>🎧 Análisis Avanzado de Llamadas</h2>"))
    display(HTML("""
        <p><b>Funcionalidades:</b></p>
        <ul>
            <li>📊 Análisis desde Excel con Caller ID</li>
            <li>🎯 Detección de verticales y flags</li>
            <li>📈 Reportes detallados</li>
            <li>🌐 Soporte multilingüe</li>
            <li>📋 Exportación a Excel</li>
        </ul>
        <p><b>Formato Excel:</b> Debe contener columna 'Recording Link' o 'Recording URL'</p>
    """))
    display(excel_input)
    display(audio_input)
    display(url_input)
    display(process_button)
    display(status_output)
    display(results_output)
    display(stats_output)
# Ejecutar la interfaz
create_ui()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileUpload(value={}, accept='.xlsx,.xls', description='Subir Excel')

FileUpload(value={}, accept='.mp3,.wav,.m4a', description='Subir Audios', multiple=True)

Textarea(value='', description='URLs:', layout=Layout(height='100px', width='600px'), placeholder='https://eje…

Button(button_style='primary', description='Procesar Todo', style=ButtonStyle())

Output()

Output()

Output()

In [ ]:
import os
import re
import logging
import pandas as pd
import torch
import numpy as np
from datetime import datetime
import requests
from faster_whisper import WhisperModel
import ipywidgets as widgets
from IPython.display import display, HTML
import glob
import uuid
import warnings
import tempfile
from concurrent.futures import ThreadPoolExecutor
warnings.filterwarnings('ignore')

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuración de Whisper
WHISPER_CONFIG = {
    "model_size": "medium",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "compute_type": "float16" if torch.cuda.is_available() else "int8",
    "language": None,  # Detectar automáticamente
    "vad_filter": True,
    "vad_parameters": {"threshold": 0.5, "min_silence_duration_ms": 500}
}

# Inicializar Whisper model una vez
try:
    WHISPER_MODEL = WhisperModel(WHISPER_CONFIG["model_size"], device=WHISPER_CONFIG["device"], compute_type=WHISPER_CONFIG["compute_type"])
except Exception as e:
    logger.error(f"Error inicializando Whisper model: {e}")
    WHISPER_MODEL = None

# Reglas de detección de verticales y flags con regex optimizadas
VERTICAL_RULES = {
    # Existing verticals can remain or be removed based on user preference
    # For now, keeping them and adding a specific 'Red Flags' focus
    "ACA": {
        "keywords": [
            r"\bhealth insurance\b", r"\baffordable care act\b", r"\bobamacare\b", r"\bmedical coverage\b",
            r"\bhealth plan\b", r"\binsurance marketplace\b", r"\bpremium tax credit\b", r"\bessential health benefits\b"
        ],
        "flags": [
            r"\b6500 gift card\b", r"\bfree gift card\b", r"\bguaranteed approval\b", r"\bno medical exam\b",
            r"\bpre-existing conditions covered\b", r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 1
    },
    "Medicare": {
        "keywords": [
            r"\bmedicare\b", r"\bmedicare advantage\b", r"\bmedicare supplement\b", r"\bmedigap\b",
            r"\bpart a\b", r"\bpart b\b", r"\bpart c\b", r"\bpart d\b", r"\bsenior benefits\b", r"\bmedicare enrollment\b"
        ],
        "flags": [
            r"\bno premium\b", r"\bzero cost\b", r"\bextra benefits\b", r"\bdental and vision included\b",
            r"\bprescription drug coverage\b", r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 2
    },
    "Debt": {
        "keywords": [
            r"\bdebt relief\b", r"\bdebt consolidation\b", r"\bdebt settlement\b", r"\bcredit card debt\b",
            r"\bpersonal loan\b", r"\bfinancial hardship\b", r"\balivio de la deuda\b", r"\balivio para sus deudas\b", r"\bdebt\b"
        ],
        "flags": [
            r"\bgovernment program\b", r"\bcancel debt\b", r"\beliminate debt\b", r"\bforgiveness program\b",
            r"\breduce debt by \d+%\b", r"\bstop paying creditors\b", r"\bprograma del gobierno\b", r"\bestan dando dinero\b", r"\bpay back\b",
            r"\bfree roofing\b", r"\bfree program\b"
        ],
        "priority": 3
    },
    "Solar": {
        "keywords": [
            r"\bsolar panels\b", r"\bsolar energy\b", r"\brenewable energy\b", r"\bsolar installation\b",
            r"\bsolar system\b", r"\bclean energy\b", r"\bsolar\b", r"\belectricity\b"
        ],
        "flags": [
            r"\bno cost solar\b", r"\bfree solar panels\b", r"\bfree solar system\b", r"\bgovernment incentive\b",
            r"\btax credit\b", r"\bzero down payment\b", r"\beliminate electric bill\b", r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 4
    },
    "Roofing": {
        "keywords": [
            r"\broof repair\b", r"\broof replacement\b", r"\broofing services\b", r"\bhome improvement\b",
            r"\bstorm damage\b"
        ],
        "flags": [
            r"\bfree roofing\b", r"\binsurance will pay\b", r"\bno out of pocket\b", r"\bgovernment program\b",
            r"\bfree program\b"
        ],
        "priority": 5
    },
    "Auto Insurance": {
        "keywords": [
            r"\bcar insurance\b", r"\bauto insurance\b", r"\bvehicle insurance\b", r"\bauto coverage\b", r"\bcomprehensive coverage\b"
        ],
        "flags": [
            r"\blower your rates\b", r"\bsave \$\d+ per month\b", r"\bcompare quotes\b", r"\bswitch and save\b",
            r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 6
    },
    "Home Security": {
        "keywords": [
            r"\bhome security\b", r"\bsecurity system\b", r"\bhome improvement\b", r"\bwindows\b", r"\bbathroom\b"
        ],
        "flags": [
            r"\bfree installation\b", r"\bfree equipment\b", r"\bgovernment discount\b", r"\bsenior discount\b",
            r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 7
    }
}

QUALITY_PATTERNS = {
    "positive_indicators": [
        r"\bthank you\b", r"\binterested\b", r"\bplease send\b", r"\btell me more\b", r"\bgracias\b", r"\binteresado\b"
    ],
    "negative_indicators": [
        r"\bnot interested\b", r"\bdo not call\b", r"\bno thanks\b", r"\bremove my number\b", r"\bno interesado\b", r"\bno llame\b"
    ],
    # Added patterns for appointment scheduling
    "appointment_indicators": [
        r"\bappointment set\b", r"\bcita programada\b", r"\bschedule a time\b", r"\bfecha\b", r"\bhora\b", r"\bvisit\b", r"\bagenda\b"
    ]
}

def transcribe(audio_path, model=WHISPER_MODEL):
    """Transcribe audio usando Whisper."""
    if model is None:
        logger.error("Whisper model no inicializado")
        return "", None, None
    try:
        segments, info = model.transcribe(
            audio_path,
            vad_filter=WHISPER_CONFIG["vad_filter"],
            vad_parameters=WHISPER_CONFIG["vad_parameters"],
            language=WHISPER_CONFIG["language"]
        )
        transcript = " ".join(segment.text for segment in segments)
        language = info.language
        confidence = info.language_probability if hasattr(info, 'language_probability') else None
        torch.cuda.empty_cache()
        return transcript, language, confidence
    except Exception as e:
        logger.error(f"Error en transcripción de {audio_path}: {e}")
        return "", None, None

def detect_vertical_and_flags(text):
    """Detecta vertical y flags en la transcripción."""
    detected_verticals = {}
    detected_flags = {}
    for vertical, config in VERTICAL_RULES.items():
        keyword_matches = []
        for pattern in config["keywords"]:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                keyword_matches.extend([pattern.replace(r'\b', '') for _ in matches])
        if keyword_matches:
            detected_verticals[vertical] = keyword_matches

        flag_matches = []
        for pattern in config["flags"]:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                flag_matches.extend([pattern.replace(r'\b', '') for _ in matches])
        if flag_matches:
            detected_flags[vertical] = flag_matches

    if detected_verticals:
        primary_vertical = min(
            detected_verticals.keys(),
            key=lambda k: (len(detected_verticals[k]), -VERTICAL_RULES[k]["priority"])
        )
        primary_flags = detected_flags.get(primary_vertical, [])
        return primary_vertical, primary_flags, detected_verticals, detected_flags
    return "No asignada", [], {}, {}

def validate_call_quality(text, flags):
    """Valida la calidad de la transcripción."""
    if not text.strip():
        return "No cumple", ["sin transcripción"]
    issues = []
    word_count = len(text.split())
    if word_count < 10:
        issues.append("transcripción muy corta")
    elif word_count < 5 and "ringing" in text.lower():
        issues.append("solo timbre")
    elif word_count < 5 and "ivr" in text.lower():
        issues.append("solo ivr")
    elif word_count < 20 and "background noise" in text.lower():
         issues.append("solo ruido de fondo")
    elif word_count < 20 and ("unresponsive" in text.lower() or "no response" in text.lower()):
        issues.append("cliente no responde")
    elif "no agent" in text.lower() or "agent not available" in text.lower():
        issues.append("sin agente")
    elif word_count < 50 and ("stops" in text.lower() or "recording ends" in text.lower()):
        issues.append("grabación interrumpida")
    for pattern in QUALITY_PATTERNS["negative_indicators"]:
        if re.search(pattern, text, re.IGNORECASE):
            issues.append(f"Contenido problemático: {pattern}")
    return "No cumple" if issues else "Cumple", issues

def generate_summary(transcript, flags, vertical):
    """Genera un resumen detallado de la transcripción."""
    summary_parts = [f"Vertical: {vertical}"]
    if flags:
        summary_parts.append(f"Flags: {', '.join(flags)}")
    else:
        summary_parts.append("Sin flags detectados")
    return ", ".join(summary_parts).strip()

def generate_transcript_summary(transcript, flags, vertical, quality_issues):
    """Genera un resumen breve de la transcripción basado en patrones y contenido."""
    if not transcript.strip():
        return "nada grabado"

    summary_parts = []

    # Check for quality issues first
    for issue in quality_issues:
        if issue == "sin transcripción":
            return "nada grabado"
        elif issue == "solo timbre":
            return "teléfono timbra, termina"
        elif issue == "solo ivr":
            return "solo ivr"
        elif issue == "solo ruido de fondo":
            return "ruido de fondo, sin agente"
        elif issue == "cliente no responde":
            return "cliente no responde tras transferencia"
        elif issue == "sin agente":
            return "sin agente, llamada termina"
        elif issue == "grabación interrumpida":
            return "grabación se detiene"
        elif issue == "transcripción muy corta":
            summary_parts.append("llamada corta")

    # Check for appointment scheduling
    appointment_scheduled = False
    for pattern in QUALITY_PATTERNS["appointment_indicators"]:
        if re.search(pattern, transcript, re.IGNORECASE):
            appointment_scheduled = True
            summary_parts.append("cita programada")
            break

    # Check for red flags (using flags detected from detect_vertical_and_flags)
    if flags:
        red_flags_found = [flag for flag in flags if any(re.search(pattern, flag, re.IGNORECASE) for pattern in [r"free roofing", r"government program", r"free program", r"misleading ad"])]
        if red_flags_found:
            summary_parts.append(f"red flags: {', '.join(red_flags_found)}")

    # Add vertical if not "No asignada" and no specific summary added yet
    if vertical != "No asignada" and not summary_parts:
         summary_parts.append(f"habló sobre {vertical.lower()}")


    if not summary_parts:
        for pattern in QUALITY_PATTERNS["negative_indicators"]:
            if re.search(pattern, transcript, re.IGNORECASE):
                summary_parts.append("no interesado")
                break
        else:
            summary_parts.append("conversación genérica")


    summary = ", ".join(summary_parts)
    words = summary.split()
    if len(words) > 15: # Adjusted length for potentially more info
        summary = " ".join(words[:15]) + "..."

    return summary.lower() if summary else "sin resumen"


def analyze_audio(audio_path, recording_url=None, publisher=None, buyer=None, caller_id=None, additional_info=None):
    try:
        transcript, language, confidence = transcribe(audio_path)
        vertical, flags, all_verticals, detected_flags = detect_vertical_and_flags(transcript)
        # Simplified vertical mapping based on initial request's focus
        vertical_map = {
            "Debt": "Spanish Debt Inbounds",
            "Roofing": "HS - RTB Roofing",
            "Home Security": "HS - RTB Bathroom", # Keeping original mapping for now
            "Solar": "Solar - Solar",
            "Auto Insurance": "Auto Insurance",
            "ACA": "ACA",
             "Medicare": "Medicare"
        }
        vertical = vertical_map.get(vertical, vertical) # Use mapped vertical, or original if not mapped
        summary = generate_summary(transcript, flags, vertical)
        compliance_status, quality_issues = validate_call_quality(transcript, flags)
        transcript_summary = generate_transcript_summary(transcript, flags, vertical, quality_issues)
        if not caller_id:
            caller_id = extract_caller_id_from_filename(os.path.basename(audio_path))
        result = {
            "Caller ID": caller_id,
            "Publisher": publisher or "N/A",
            "Buyer": buyer or "N/A",
            "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Recording Link": recording_url or audio_path,
            "Vertical": vertical,
            "Flags Found": ", ".join(flags) if flags else "Ninguna",
            "Flags Count": len(flags),
            "Summary": summary, # This is the detailed summary
            "Compliance": compliance_status,
            "Language": language,
            "Language Confidence": f"{confidence:.2f}" if confidence else "N/A",
            "Transcript Length": len(transcript.split()),
            "Full Transcript": transcript,
            "Quality Issues": ", ".join(quality_issues) if quality_issues else "None",
            "Additional Info": str(additional_info) if additional_info else "N/A",
            "Transcript Summary": transcript_summary # This is the brief summary with flags/appointment
        }
        return result
    except Exception as e:
        logger.error(f"Error analyzing audio {audio_path}: {e}")
        return {
            "Caller ID": caller_id or "Error",
            "Publisher": publisher or "N/A",
            "Buyer": buyer or "N/A",
            "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Recording Link": recording_url or audio_path,
            "Vertical": "Error",
            "Flags Found": "Error",
            "Flags Count": 0,
            "Summary": f"Error en análisis: {str(e)}",
            "Compliance": "Error",
            "Language": "Error",
            "Language Confidence": "N/A",
            "Transcript Length": 0,
            "Full Transcript": f"Error: {str(e)}",
            "Quality Issues": str(e),
            "Additional Info": "Error en procesamiento",
            "Transcript Summary": "Error en transcripción"
        }

def extract_caller_id_from_filename(filename):
    """Extract caller ID from filename if possible."""
    match = re.search(r'\d{10}', filename)
    return match.group(0) if match else "Unknown"

def process_url(url, publisher="N/A", buyer="N/A", caller_id=None):
    """Procesa una URL de audio."""
    try:
        response = requests.get(url, stream=True, timeout=10)
        if response.status_code == 200:
            # Infer file extension from URL if possible
            url_path = requests.utils.urlparse(url).path
            ext = os.path.splitext(url_path)[1] if os.path.splitext(url_path)[1] else '.mp3' # Default to .mp3
            with tempfile.NamedTemporaryFile(suffix=ext, delete=False) as temp_file:
                for chunk in response.iter_content(chunk_size=8192):
                    temp_file.write(chunk)
                temp_path = temp_file.name
            try:
                result = analyze_audio(temp_path, recording_url=url, publisher=publisher, buyer=buyer, caller_id=caller_id)
            finally:
                os.remove(temp_path)
            return result
        else:
            return {
                "Caller ID": caller_id or "Error",
                "Publisher": publisher,
                "Buyer": buyer,
                "Recording Link": url,
                "Vertical": "Error",
                "Flags Found": "Error de descarga",
                "Flags Count": 0,
                "Summary": "Error descargando audio",
                "Compliance": "Error",
                "Language": "Error",
                "Language Confidence": "N/A",
                "Transcript Length": 0,
                "Full Transcript": "Error de descarga",
                "Quality Issues": "Error de descarga",
                "Additional Info": "N/A",
                "Transcript Summary": "Error en descarga"
            }
    except Exception as e:
        logger.error(f"Error procesando URL {url}: {e}")
        return {
            "Caller ID": caller_id or "Error",
            "Publisher": publisher,
            "Buyer": buyer,
            "Recording Link": url,
            "Vertical": "Error",
            "Flags Found": "Error de procesamiento",
            "Flags Count": 0,
            "Summary": f"Error: {str(e)}",
            "Compliance": "Error",
            "Language": "Error",
            "Language Confidence": "N/A",
            "Transcript Length": 0,
            "Full Transcript": f"Error: {str(e)}",
            "Quality Issues": str(e),
            "Additional Info": "N/A",
            "Transcript Summary": "Error en descarga"
        }


def process_mixed_inputs(audio_files, url_input, excel_file, progress_callback=None):
    """Procesa una mezcla de audios, URLs y archivos Excel."""
    start_time = datetime.now()
    results = []
    total_items = len(audio_files) + (1 if excel_file else 0) + (len([url.strip() for url in url_input.split(',') if url.strip()]) if url_input else 0)
    processed_items = 0

    for audio_file in audio_files:
        result = analyze_audio(audio_file.name)
        results.append(result)
        processed_items += 1
        if progress_callback:
            progress_callback(processed_items / total_items, f"Procesando audio: {os.path.basename(audio_file.name)}")

    if url_input:
        urls = [url.strip() for url in url_input.split(',') if url.strip()]
        with ThreadPoolExecutor(max_workers=5) as executor:
            # Map returns results in the order the tasks were submitted
            url_results = list(executor.map(process_url, urls))
            results.extend(url_results)
        processed_items += len(urls)
        if progress_callback:
            progress_callback(processed_items / total_items, f"Procesando URLs")


    if excel_file:
        try:
            df_excel = pd.read_excel(excel_file)
            # Use the enhanced mapping function to find columns
            found_columns = {}
            for key, possible_names in EXCEL_COLUMN_MAPPING.items():
                for col_name in possible_names:
                    if col_name in df_excel.columns:
                        found_columns[key] = col_name
                        break

            url_column = found_columns.get('recording_link')
            if not url_column:
                 raise ValueError(f"El archivo Excel debe contener una columna de URL válida. Columnas buscadas: {EXCEL_COLUMN_MAPPING['recording_link']}")

            processing_info = []
            for idx, row in df_excel.iterrows():
                 url = str(row.get(url_column, '')).strip()
                 if url: # Only process rows with a valid URL
                    info = {
                        'url': url,
                        'row_index': idx,
                        'data': {key: str(row[col]).strip() for key, col in found_columns.items() if col in row and pd.notna(row[col])}
                    }
                    additional_info = {col: str(row[col]).strip() for col in df_excel.columns if col not in found_columns.values() and col in row and pd.notna(row[col])}
                    if additional_info:
                        info['data']['additional'] = additional_info
                    processing_info.append(info)


            def process_excel_row(item):
                url = item['url']
                row_data = item['data']
                caller_id = row_data.get('caller_id') or extract_caller_id_from_filename(os.path.basename(url)) # Extract from filename if not in excel
                return process_url(
                    url,
                    publisher=row_data.get('publisher'),
                    buyer=row_data.get('buyer'),
                    caller_id=caller_id
                    # Note: additional_info is not passed to process_url/analyze_audio in the current structure
                    # If needed, the analyze_audio signature would need to be adjusted to handle 'data' dict
                )

            with ThreadPoolExecutor(max_workers=5) as executor:
                excel_results = list(executor.map(process_excel_row, processing_info))
                results.extend(excel_results)

            processed_items += len(processing_info)
            if progress_callback:
                progress_callback(processed_items / total_items, f"Procesando Excel")

        except Exception as e:
            logger.error(f"Error procesando Excel {excel_file}: {e}")
            results.append({
                "Caller ID": "Error",
                "Recording Link": excel_file,
                "Vertical": "Error",
                "Summary": f"Error en Excel: {str(e)}",
                "Transcript Summary": "Error en Excel",
                "Flags Found": "Error",
                "Flags Count": 0,
                "Compliance": "Error",
                "Language": "Error",
                "Language Confidence": "N/A",
                "Transcript Length": 0,
                "Full Transcript": f"Error: {str(e)}",
                "Quality Issues": str(e),
                "Additional Info": "Error en procesamiento"
            })


    df_results = pd.DataFrame(results)
    total_time = (datetime.now() - start_time).total_seconds()
    return df_results, total_time

def generate_statistics_markdown(df, total_time):
    """Genera estadísticas en formato markdown."""
    total_calls = len(df)
    if total_calls == 0:
        return "Sin datos para mostrar estadísticas"

    # Ensure columns exist before accessing
    flags_count_col = df.get('Flags Count', pd.Series(dtype=int))
    compliance_col = df.get('Compliance', pd.Series(dtype=str))
    vertical_col = df.get('Vertical', pd.Series(dtype=str))

    calls_with_flags = len(flags_count_col[flags_count_col > 0])
    compliance_rate = len(compliance_col[compliance_col == 'Cumple']) / total_calls * 100 if total_calls > 0 else 0
    vertical_counts = vertical_col.value_counts().to_dict()

    # Calculate time in minutes and seconds
    minutes = int(total_time // 60)
    seconds = int(total_time % 60)
    time_str = f"{minutes} min {seconds} seg" if minutes > 0 else f"{seconds} seg"


    stats = f"""
## 📊 Resumen de Análisis
- **Total de Llamadas**: {total_calls}
- **Llamadas con Marcas/Flags**: {calls_with_flags} ({calls_with_flags/total_calls*100:.1f}%)
- **Tasa de Cumplimiento**: {compliance_rate:.1f}%
- **Tiempo Total**: {time_str}

### Distribución por Vertical
"""
    for vertical, count in vertical_counts.items():
        stats += f"- {vertical}: {count} ({count/total_calls*100:.1f}%)\n"

    # Add flags count if 'Flags Found' column exists
    if 'Flags Found' in df.columns:
        all_flags = []
        # Iterate through the Series and handle potential NaN or non-string values
        for flags_str in df['Flags Found'].dropna():
            if isinstance(flags_str, str) and flags_str != 'Ninguna' and flags_str != 'Error':
                 all_flags.extend([f.strip() for f in flags_str.split(',')])

        if all_flags:
            flag_counts = pd.Series(all_flags).value_counts().head(10) # Show top 10 flags
            stats += "\n### Top Flags Encontrados\n"
            for flag, count in flag_counts.items():
                stats += f"- **{flag}**: {count} veces\n"

    # Add appointment count if 'Transcript Summary' column exists and contains "cita programada"
    if 'Transcript Summary' in df.columns:
        appointment_count = len(df[df['Transcript Summary'].str.contains('cita programada', na=False)])
        stats += f"\n- **Llamadas con Cita Programada**: {appointment_count}\n"


    return stats

def export_enhanced_excel(df):
    """Exporta los resultados a un archivo Excel con formato mejorado."""
    try:
        # Exporting to /content directory
        output_path = f'/content/call_analysis_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
        df.to_excel(output_path, index=False, engine='openpyxl')
        logger.info(f"Excel exportado a {output_path}")
        return output_path
    except Exception as e:
        logger.error(f"Error exportando Excel: {e}")
        return None

# Keep the EXCEL_COLUMN_MAPPING from the first cell
EXCEL_COLUMN_MAPPING = {
    'recording_link': [
        'Recording Link', 'Recording URL', 'recording_url', 'URL', 'url',
        'Audio URL', 'Link', 'recording_link', 'audio_link', 'call_recording'
    ],
    'caller_id': [
        'Caller ID', 'CallID', 'caller_id', 'ID', 'Phone', 'Caller',
        'phone_number', 'PhoneNumber', 'lead_phone', 'customer_phone', 'From' # Added 'From'
    ],
    'publisher': ['Publisher', 'publisher', 'Pub', 'Source', 'Traffic Source', 'Partner'],
    'buyer': ['Buyer', 'buyer', 'Client', 'Customer', 'Account'],
    'date': ['Date', 'date', 'Call Date', 'Timestamp', 'Created Date', 'call_date'],
    'time': ['Time', 'time', 'Call Time', 'Duration', 'Call Duration', 'Length'],
    'campaign': ['Campaign', 'campaign', 'Campaign Name', 'Campaign ID'],
    'lead_id': ['Lead ID', 'lead_id', 'LeadID', 'Lead', 'Lead Number'],
    'vertical': ['Vertical', 'vertical', 'Industry', 'Category', 'Product'],
    'status': ['Status', 'status', 'Call Status', 'Disposition']
}


def create_ui():
    excel_input = widgets.FileUpload(accept='.xlsx,.xls', multiple=False, description='Subir Excel')
    audio_input = widgets.FileUpload(accept='.mp3,.wav,.m4a', multiple=True, description='Subir Audios')
    url_input = widgets.Textarea(value='', placeholder='https://ejemplo.com/audio1.mp3, https://ejemplo.com/audio2.mp3', description='URLs:', layout={'width': '600px', 'height': '100px'})
    process_button = widgets.Button(description='Procesar Todo', button_style='primary')
    progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progreso:', bar_style='info', layout={'width': '600px'})
    status_label = widgets.Label(value='Listo para procesar')
    status_output = widgets.Output()
    results_output = widgets.Output()
    stats_output = widgets.Output()

    def process_all_inputs(b):
        with status_output:
            status_output.clear_output()
            results_output.clear_output()
            stats_output.clear_output()
            progress_bar.value = 0.0
            status_label.value = "⏳ Iniciando procesamiento..."

        files = []
        for filename, file_info in audio_input.value.items():
            if not filename.lower().endswith(('.mp3', '.wav', '.m4a')):
                with status_output:
                    print(f"❌ Archivo {filename} no soportado")
                continue
            with tempfile.NamedTemporaryFile(suffix=os.path.splitext(filename)[1], delete=False) as temp_file:
                temp_file.write(file_info['content'])
                files.append(type('obj', (object,), {'name': temp_file.name}))

        excel_file = None
        if excel_input.value:
            filename = list(excel_input.value.keys())[0]
            if not filename.lower().endswith(('.xlsx', '.xls')):
                with status_output:
                    status_output.clear_output()
                    status_label.value = f"❌ Archivo {filename} no es un Excel válido"
                return
            with tempfile.NamedTemporaryFile(suffix='.xlsx', delete=False) as temp_file:
                temp_file.write(excel_input.value[filename]['content'])
                excel_file = temp_file.name

        def progress_callback(progress, message):
            with status_output:
                status_output.clear_output()
                progress_bar.value = progress
                status_label.value = f"{message} ({progress*100:.1f}%)"

        try:
            df, total_time = process_mixed_inputs(files, url_input.value, excel_file, progress_callback)
            if len(df) == 0:
                with status_output:
                    status_output.clear_output()
                    progress_bar.value = 0.0
                    status_label.value = "❌ No se encontraron datos para procesar"
                # Clean up temp files
                for temp_file in files + ([excel_file] if excel_file else []):
                    if temp_file and os.path.exists(temp_file.name if hasattr(temp_file, 'name') else temp_file):
                        try:
                            os.remove(temp_file.name if hasattr(temp_file, 'name') else temp_file)
                        except Exception as e:
                            logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")
                # Reset widgets
                excel_input.value.clear()
                audio_input.value.clear()
                url_input.value = ''
                excel_input._counter = 0
                audio_input._counter = 0
                return

            # Ensure necessary columns for preview exist, add if not
            preview_cols_base = ["Caller ID", "Vertical", "Flags Found", "Transcript Summary", "Compliance"]
            preview_cols = [col for col in preview_cols_base if col in df.columns]

            preview_df = df[preview_cols].head(50)
            stats_md = generate_statistics_markdown(df, total_time)
            excel_path = export_enhanced_excel(df)

            with results_output:
                results_output.clear_output()
                display(HTML("<h3>Resultados (Primeras 50 Filas)</h3>"))
                display(preview_df)

            with stats_output:
                stats_output.clear_output()
                display(HTML("<h3>Estadísticas</h3>"))
                display(HTML(stats_md.replace('\n', '<br>')))

            with status_output:
                status_output.clear_output()
                progress_bar.value = 1.0
                if excel_path:
                    status_label.value = f"✅ Procesamiento completado: {len(df)} audios analizados"
                    display(HTML(f'<p>Reporte Excel guardado en: <b>{excel_path}</b></p>'))
                    display(HTML(f'<a href="{excel_path}" download>Descargar Excel</a>'))
                else:
                    status_label.value = f"⚠️ Procesados {len(df)} audios, pero error en exportar Excel"
                    display(HTML("<p>Hubo un error al exportar el archivo Excel. Verifique los logs para más detalles.</p>"))


            # Clean up temp files after processing
            for temp_file in files + ([excel_file] if excel_file else []):
                 if temp_file and os.path.exists(temp_file.name if hasattr(temp_file, 'name') else temp_file):
                    try:
                        os.remove(temp_file.name if hasattr(temp_file, 'name') else temp_file)
                    except Exception as e:
                        logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")

            # Reset widgets
            excel_input.value.clear()
            audio_input.value.clear()
            url_input.value = ''
            excel_input._counter = 0
            audio_input._counter = 0


        except Exception as e:
            with status_output:
                status_output.clear_output()
                progress_bar.value = 0.0
                status_label.value = f"❌ Error: {str(e)}"
            # Clean up temp files in case of error
            for temp_file in files + ([excel_file] if excel_file else []):
                if temp_file and os.path.exists(temp_file.name if hasattr(temp_file, 'name') else temp_file):
                    try:
                        os.remove(temp_file.name if hasattr(temp_file, 'name') else temp_file)
                    except Exception as e:
                        logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")
             # Reset widgets
            excel_input.value.clear()
            audio_input.value.clear()
            url_input.value = ''
            excel_input._counter = 0
            audio_input._counter = 0



    process_button.on_click(process_all_inputs)
    display(HTML("<h2>🎧 Análisis Avanzado de Llamadas</h2>"))
    display(HTML("""
        <p><b>Funcionalidades:</b></p>
        <ul>
            <li>📊 Análisis desde Excel con URLs de Grabación</li>
            <li>🎯 Detección de Flags (Ej: 'free roofing', 'government program')</li>
            <li>📅 Detección de Citas Programadas</li>
            <li>📈 Reportes detallados en Excel y Resumen en Notebook</li>
            <li>🌐 Soporte multilingüe</li>
        </ul>
        <p><b>Formato Excel:</b> Debe contener una columna con las URLs de grabación (Ej: 'Recording Link', 'Recording URL'). Columnas opcionales para metadatos: 'Publisher', 'Buyer', 'Caller ID', 'From'.</p>
    """))
    display(excel_input)
    display(audio_input)
    display(url_input)
    display(process_button)
    display(progress_bar)
    display(status_label)
    display(status_output)
    display(results_output)
    display(stats_output)

# Ensure EXCEL_COLUMN_MAPPING is defined before calling create_ui
EXCEL_COLUMN_MAPPING = {
    'recording_link': [
        'Recording Link', 'Recording URL', 'recording_url', 'URL', 'url',
        'Audio URL', 'Link', 'recording_link', 'audio_link', 'call_recording'
    ],
    'caller_id': [
        'Caller ID', 'CallID', 'caller_id', 'ID', 'Phone', 'Caller',
        'phone_number', 'PhoneNumber', 'lead_phone', 'customer_phone', 'From'
    ],
    'publisher': ['Publisher', 'publisher', 'Pub', 'Source', 'Traffic Source', 'Partner'],
    'buyer': ['Buyer', 'buyer', 'Client', 'Customer', 'Account'],
    'date': ['Date', 'date', 'Call Date', 'Timestamp', 'Created Date', 'call_date'],
    'time': ['Time', 'time', 'Call Time', 'Duration', 'Call Duration', 'Length'],
    'campaign': ['Campaign', 'campaign', 'Campaign Name', 'Campaign ID'],
    'lead_id': ['Lead ID', 'lead_id', 'LeadID', 'Lead', 'Lead Number'],
    'vertical': ['Vertical', 'vertical', 'Industry', 'Category', 'Product'],
    'status': ['Status', 'status', 'Call Status', 'Disposition']
}


create_ui()

ModuleNotFoundError: No module named 'faster_whisper'

In [ ]:
!pip install faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
!pip install faster-whisper
import os
import re
import logging
import pandas as pd
import torch
import numpy as np
from datetime import datetime
import requests
from faster_whisper import WhisperModel
import ipywidgets as widgets
from IPython.display import display, HTML
import glob
import uuid
import warnings
import tempfile
from concurrent.futures import ThreadPoolExecutor
warnings.filterwarnings('ignore')

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuración de Whisper
WHISPER_CONFIG = {
    "model_size": "medium",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "compute_type": "float16" if torch.cuda.is_available() else "int8",
    "language": None,  # Detectar automáticamente
    "vad_filter": True,
    "vad_parameters": {"threshold": 0.5, "min_silence_duration_ms": 500}
}

# Inicializar Whisper model una vez
try:
    WHISPER_MODEL = WhisperModel(WHISPER_CONFIG["model_size"], device=WHISPER_CONFIG["device"], compute_type=WHISPER_CONFIG["compute_type"])
except Exception as e:
    logger.error(f"Error inicializando Whisper model: {e}")
    WHISPER_MODEL = None

# Reglas de detección de verticales y flags con regex optimizadas
VERTICAL_RULES = {
    # Existing verticals can remain or be removed based on user preference
    # For now, keeping them and adding a specific 'Red Flags' focus
    "ACA": {
        "keywords": [
            r"\bhealth insurance\b", r"\baffordable care act\b", r"\bobamacare\b", r"\bmedical coverage\b",
            r"\bhealth plan\b", r"\binsurance marketplace\b", r"\bpremium tax credit\b", r"\bessential health benefits\b"
        ],
        "flags": [
            r"\b6500 gift card\b", r"\bfree gift card\b", r"\bguaranteed approval\b", r"\bno medical exam\b",
            r"\bpre-existing conditions covered\b", r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 1
    },
    "Medicare": {
        "keywords": [
            r"\bmedicare\b", r"\bmedicare advantage\b", r"\bmedicare supplement\b", r"\bmedigap\b",
            r"\bpart a\b", r"\bpart b\b", r"\bpart c\b", r"\bpart d\b", r"\bsenior benefits\b", r"\bmedicare enrollment\b"
        ],
        "flags": [
            r"\bno premium\b", r"\bzero cost\b", r"\bextra benefits\b", r"\bdental and vision included\b",
            r"\bprescription drug coverage\b", r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 2
    },
    "Debt": {
        "keywords": [
            r"\bdebt relief\b", r"\bdebt consolidation\b", r"\bdebt settlement\b", r"\bcredit card debt\b",
            r"\bpersonal loan\b", r"\bfinancial hardship\b", r"\balivio de la deuda\b", r"\balivio para sus deudas\b", r"\bdebt\b"
        ],
        "flags": [
            r"\bgovernment program\b", r"\bcancel debt\b", r"\beliminate debt\b", r"\bforgiveness program\b",
            r"\breduce debt by \d+%\b", r"\bstop paying creditors\b", r"\bprograma del gobierno\b", r"\bestan dando dinero\b", r"\bpay back\b",
            r"\bfree roofing\b", r"\bfree program\b"
        ],
        "priority": 3
    },
    "Solar": {
        "keywords": [
            r"\bsolar panels\b", r"\bsolar energy\b", r"\brenewable energy\b", r"\bsolar installation\b",
            r"\bsolar system\b", r"\bclean energy\b", r"\bsolar\b", r"\belectricity\b"
        ],
        "flags": [
            r"\bno cost solar\b", r"\bfree solar panels\b", r"\bfree solar system\b", r"\bgovernment incentive\b",
            r"\btax credit\b", r"\bzero down payment\b", r"\beliminate electric bill\b", r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 4
    },
    "Roofing": {
        "keywords": [
            r"\broof repair\b", r"\broof replacement\b", r"\broofing services\b", r"\bhome improvement\b",
            r"\bstorm damage\b"
        ],
        "flags": [
            r"\bfree roofing\b", r"\binsurance will pay\b", r"\bno out of pocket\b", r"\bgovernment program\b",
            r"\bfree program\b"
        ],
        "priority": 5
    },
    "Auto Insurance": {
        "keywords": [
            r"\bcar insurance\b", r"\bauto insurance\b", r"\bvehicle insurance\b", r"\bauto coverage\b", r"\bcomprehensive coverage\b"
        ],
        "flags": [
            r"\blower your rates\b", r"\bsave \$\d+ per month\b", r"\bcompare quotes\b", r"\bswitch and save\b",
            r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 6
    },
    "Home Security": {
        "keywords": [
            r"\bhome security\b", r"\bsecurity system\b", r"\bhome improvement\b", r"\bwindows\b", r"\bbathroom\b"
        ],
        "flags": [
            r"\bfree installation\b", r"\bfree equipment\b", r"\bgovernment discount\b", r"\bsenior discount\b",
            r"\bfree roofing\b", r"\bgovernment program\b", r"\bfree program\b"
        ],
        "priority": 7
    }
}

QUALITY_PATTERNS = {
    "positive_indicators": [
        r"\bthank you\b", r"\binterested\b", r"\bplease send\b", r"\btell me more\b", r"\bgracias\b", r"\binteresado\b"
    ],
    "negative_indicators": [
        r"\bnot interested\b", r"\bdo not call\b", r"\bno thanks\b", r"\bremove my number\b", r"\bno interesado\b", r"\bno llame\b"
    ],
    # Added patterns for appointment scheduling
    "appointment_indicators": [
        r"\bappointment set\b", r"\bcita programada\b", r"\bschedule a time\b", r"\bfecha\b", r"\bhora\b", r"\bvisit\b", r"\bagenda\b"
    ]
}

def transcribe(audio_path, model=WHISPER_MODEL):
    """Transcribe audio usando Whisper."""
    if model is None:
        logger.error("Whisper model no inicializado")
        return "", None, None
    try:
        segments, info = model.transcribe(
            audio_path,
            vad_filter=WHISPER_CONFIG["vad_filter"],
            vad_parameters=WHISPER_CONFIG["vad_parameters"],
            language=WHISPER_CONFIG["language"]
        )
        transcript = " ".join(segment.text for segment in segments)
        language = info.language
        confidence = info.language_probability if hasattr(info, 'language_probability') else None
        torch.cuda.empty_cache()
        return transcript, language, confidence
    except Exception as e:
        logger.error(f"Error en transcripción de {audio_path}: {e}")
        return "", None, None

def detect_vertical_and_flags(text):
    """Detecta vertical y flags en la transcripción."""
    detected_verticals = {}
    detected_flags = {}
    for vertical, config in VERTICAL_RULES.items():
        keyword_matches = []
        for pattern in config["keywords"]:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                keyword_matches.extend([pattern.replace(r'\b', '') for _ in matches])
        if keyword_matches:
            detected_verticals[vertical] = keyword_matches

        flag_matches = []
        for pattern in config["flags"]:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                flag_matches.extend([pattern.replace(r'\b', '') for _ in matches])
        if flag_matches:
            detected_flags[vertical] = flag_matches

    if detected_verticals:
        primary_vertical = min(
            detected_verticals.keys(),
            key=lambda k: (len(detected_verticals[k]), -VERTICAL_RULES[k]["priority"])
        )
        primary_flags = detected_flags.get(primary_vertical, [])
        return primary_vertical, primary_flags, detected_verticals, detected_flags
    return "No asignada", [], {}, {}

def validate_call_quality(text, flags):
    """Valida la calidad de la transcripción."""
    if not text.strip():
        return "No cumple", ["sin transcripción"]
    issues = []
    word_count = len(text.split())
    if word_count < 10:
        issues.append("transcripción muy corta")
    elif word_count < 5 and "ringing" in text.lower():
        issues.append("solo timbre")
    elif word_count < 5 and "ivr" in text.lower():
        issues.append("solo ivr")
    elif word_count < 20 and "background noise" in text.lower():
         issues.append("solo ruido de fondo")
    elif word_count < 20 and ("unresponsive" in text.lower() or "no response" in text.lower()):
        issues.append("cliente no responde")
    elif "no agent" in text.lower() or "agent not available" in text.lower():
        issues.append("sin agente")
    elif word_count < 50 and ("stops" in text.lower() or "recording ends" in text.lower()):
        issues.append("grabación interrumpida")
    for pattern in QUALITY_PATTERNS["negative_indicators"]:
        if re.search(pattern, text, re.IGNORECASE):
            issues.append(f"Contenido problemático: {pattern}")
    return "No cumple" if issues else "Cumple", issues

def generate_summary(transcript, flags, vertical):
    """Genera un resumen detallado de la transcripción."""
    summary_parts = [f"Vertical: {vertical}"]
    if flags:
        summary_parts.append(f"Flags: {', '.join(flags)}")
    else:
        summary_parts.append("Sin flags detectados")
    return ", ".join(summary_parts).strip()

def generate_transcript_summary(transcript, flags, vertical, quality_issues):
    """Genera un resumen breve de la transcripción basado en patrones y contenido."""
    if not transcript.strip():
        return "nada grabado"

    summary_parts = []

    # Check for quality issues first
    for issue in quality_issues:
        if issue == "sin transcripción":
            return "nada grabado"
        elif issue == "solo timbre":
            return "teléfono timbra, termina"
        elif issue == "solo ivr":
            return "solo ivr"
        elif issue == "solo ruido de fondo":
            return "ruido de fondo, sin agente"
        elif issue == "cliente no responde":
            return "cliente no responde tras transferencia"
        elif issue == "sin agente":
            return "sin agente, llamada termina"
        elif issue == "grabación interrumpida":
            return "grabación se detiene"
        elif issue == "transcripción muy corta":
            summary_parts.append("llamada corta")

    # Check for appointment scheduling
    appointment_scheduled = False
    for pattern in QUALITY_PATTERNS["appointment_indicators"]:
        if re.search(pattern, transcript, re.IGNORECASE):
            appointment_scheduled = True
            summary_parts.append("cita programada")
            break

    # Check for red flags (using flags detected from detect_vertical_and_flags)
    if flags:
        red_flags_found = [flag for flag in flags if any(re.search(pattern, flag, re.IGNORECASE) for pattern in [r"free roofing", r"government program", r"free program", r"misleading ad"])]
        if red_flags_found:
            summary_parts.append(f"red flags: {', '.join(red_flags_found)}")

    # Add vertical if not "No asignada" and no specific summary added yet
    if vertical != "No asignada" and not summary_parts:
         summary_parts.append(f"habló sobre {vertical.lower()}")


    if not summary_parts:
        for pattern in QUALITY_PATTERNS["negative_indicators"]:
            if re.search(pattern, transcript, re.IGNORECASE):
                summary_parts.append("no interesado")
                break
        else:
            summary_parts.append("conversación genérica")


    summary = ", ".join(summary_parts)
    words = summary.split()
    if len(words) > 15: # Adjusted length for potentially more info
        summary = " ".join(words[:15]) + "..."

    return summary.lower() if summary else "sin resumen"


def analyze_audio(audio_path, recording_url=None, publisher=None, buyer=None, caller_id=None, additional_info=None):
    try:
        transcript, language, confidence = transcribe(audio_path)
        vertical, flags, all_verticals, detected_flags = detect_vertical_and_flags(transcript)
        # Simplified vertical mapping based on initial request's focus
        vertical_map = {
            "Debt": "Spanish Debt Inbounds",
            "Roofing": "HS - RTB Roofing",
            "Home Security": "HS - RTB Bathroom", # Keeping original mapping for now
            "Solar": "Solar - Solar",
            "Auto Insurance": "Auto Insurance",
            "ACA": "ACA",
             "Medicare": "Medicare"
        }
        vertical = vertical_map.get(vertical, vertical) # Use mapped vertical, or original if not mapped
        summary = generate_summary(transcript, flags, vertical)
        compliance_status, quality_issues = validate_call_quality(transcript, flags)
        transcript_summary = generate_transcript_summary(transcript, flags, vertical, quality_issues)
        if not caller_id:
            caller_id = extract_caller_id_from_filename(os.path.basename(audio_path))
        result = {
            "Caller ID": caller_id,
            "Publisher": publisher or "N/A",
            "Buyer": buyer or "N/A",
            "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Recording Link": recording_url or audio_path,
            "Vertical": vertical,
            "Flags Found": ", ".join(flags) if flags else "Ninguna",
            "Flags Count": len(flags),
            "Summary": summary, # This is the detailed summary
            "Compliance": compliance_status,
            "Language": language,
            "Language Confidence": f"{confidence:.2f}" if confidence else "N/A",
            "Transcript Length": len(transcript.split()),
            "Full Transcript": transcript,
            "Quality Issues": ", ".join(quality_issues) if quality_issues else "None",
            "Additional Info": str(additional_info) if additional_info else "N/A",
            "Transcript Summary": transcript_summary # This is the brief summary with flags/appointment
        }
        return result
    except Exception as e:
        logger.error(f"Error analyzing audio {audio_path}: {e}")
        return {
            "Caller ID": caller_id or "Error",
            "Publisher": publisher or "N/A",
            "Buyer": buyer or "N/A",
            "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Recording Link": recording_url or audio_path,
            "Vertical": "Error",
            "Flags Found": "Error",
            "Flags Count": 0,
            "Summary": f"Error en análisis: {str(e)}",
            "Compliance": "Error",
            "Language": "Error",
            "Language Confidence": "N/A",
            "Transcript Length": 0,
            "Full Transcript": f"Error: {str(e)}",
            "Quality Issues": str(e),
            "Additional Info": "Error en procesamiento",
            "Transcript Summary": "Error en transcripción"
        }

def extract_caller_id_from_filename(filename):
    """Extract caller ID from filename if possible."""
    match = re.search(r'\d{10}', filename)
    return match.group(0) if match else "Unknown"

def process_url(url, publisher="N/A", buyer="N/A", caller_id=None):
    """Procesa una URL de audio."""
    try:
        response = requests.get(url, stream=True, timeout=10)
        if response.status_code == 200:
            # Infer file extension from URL if possible
            url_path = requests.utils.urlparse(url).path
            ext = os.path.splitext(url_path)[1] if os.path.splitext(url_path)[1] else '.mp3' # Default to .mp3
            with tempfile.NamedTemporaryFile(suffix=ext, delete=False) as temp_file:
                for chunk in response.iter_content(chunk_size=8192):
                    temp_file.write(chunk)
                temp_path = temp_file.name
            try:
                result = analyze_audio(temp_path, recording_url=url, publisher=publisher, buyer=buyer, caller_id=caller_id)
            finally:
                os.remove(temp_path)
            return result
        else:
            return {
                "Caller ID": caller_id or "Error",
                "Publisher": publisher,
                "Buyer": buyer,
                "Recording Link": url,
                "Vertical": "Error",
                "Flags Found": "Error de descarga",
                "Flags Count": 0,
                "Summary": "Error descargando audio",
                "Compliance": "Error",
                "Language": "Error",
                "Language Confidence": "N/A",
                "Transcript Length": 0,
                "Full Transcript": "Error de descarga",
                "Quality Issues": "Error de descarga",
                "Additional Info": "N/A",
                "Transcript Summary": "Error en descarga"
            }
    except Exception as e:
        logger.error(f"Error procesando URL {url}: {e}")
        return {
            "Caller ID": caller_id or "Error",
            "Publisher": publisher,
            "Buyer": buyer,
            "Recording Link": url,
            "Vertical": "Error",
            "Flags Found": "Error de procesamiento",
            "Flags Count": 0,
            "Summary": f"Error: {str(e)}",
            "Compliance": "Error",
            "Language": "Error",
            "Language Confidence": "N/A",
            "Transcript Length": 0,
            "Full Transcript": f"Error: {str(e)}",
            "Quality Issues": str(e),
            "Additional Info": "N/A",
            "Transcript Summary": "Error en descarga"
        }


def process_mixed_inputs(audio_files, url_input, excel_file, progress_callback=None):
    """Procesa una mezcla de audios, URLs y archivos Excel."""
    start_time = datetime.now()
    results = []
    total_items = len(audio_files) + (1 if excel_file else 0) + (len([url.strip() for url in url_input.split(',') if url.strip()]) if url_input else 0)
    processed_items = 0

    for audio_file in audio_files:
        result = analyze_audio(audio_file.name)
        results.append(result)
        processed_items += 1
        if progress_callback:
            progress_callback(processed_items / total_items, f"Procesando audio: {os.path.basename(audio_file.name)}")

    if url_input:
        urls = [url.strip() for url in url_input.split(',') if url.strip()]
        with ThreadPoolExecutor(max_workers=5) as executor:
            # Map returns results in the order the tasks were submitted
            url_results = list(executor.map(process_url, urls))
            results.extend(url_results)
        processed_items += len(urls)
        if progress_callback:
            progress_callback(processed_items / total_items, f"Procesando URLs")


    if excel_file:
        try:
            df_excel = pd.read_excel(excel_file)
            # Use the enhanced mapping function to find columns
            found_columns = {}
            for key, possible_names in EXCEL_COLUMN_MAPPING.items():
                for col_name in possible_names:
                    if col_name in df_excel.columns:
                        found_columns[key] = col_name
                        break

            url_column = found_columns.get('recording_link')
            if not url_column:
                 raise ValueError(f"El archivo Excel debe contener una columna de URL válida. Columnas buscadas: {EXCEL_COLUMN_MAPPING['recording_link']}")

            processing_info = []
            for idx, row in df_excel.iterrows():
                 url = str(row.get(url_column, '')).strip()
                 if url: # Only process rows with a valid URL
                    info = {
                        'url': url,
                        'row_index': idx,
                        'data': {key: str(row[col]).strip() for key, col in found_columns.items() if col in row and pd.notna(row[col])}
                    }
                    additional_info = {col: str(row[col]).strip() for col in df_excel.columns if col not in found_columns.values() and col in row and pd.notna(row[col])}
                    if additional_info:
                        info['data']['additional'] = additional_info
                    processing_info.append(info)


            def process_excel_row(item):
                url = item['url']
                row_data = item['data']
                caller_id = row_data.get('caller_id') or extract_caller_id_from_filename(os.path.basename(url)) # Extract from filename if not in excel
                return process_url(
                    url,
                    publisher=row_data.get('publisher'),
                    buyer=row_data.get('buyer'),
                    caller_id=caller_id
                    # Note: additional_info is not passed to process_url/analyze_audio in the current structure
                    # If needed, the analyze_audio signature would need to be adjusted to handle 'data' dict
                )

            with ThreadPoolExecutor(max_workers=5) as executor:
                excel_results = list(executor.map(process_excel_row, processing_info))
                results.extend(excel_results)

            processed_items += len(processing_info)
            if progress_callback:
                progress_callback(processed_items / total_items, f"Procesando Excel")

        except Exception as e:
            logger.error(f"Error procesando Excel {excel_file}: {e}")
            results.append({
                "Caller ID": "Error",
                "Recording Link": excel_file,
                "Vertical": "Error",
                "Summary": f"Error en Excel: {str(e)}",
                "Transcript Summary": "Error en Excel",
                "Flags Found": "Error",
                "Flags Count": 0,
                "Compliance": "Error",
                "Language": "Error",
                "Language Confidence": "N/A",
                "Transcript Length": 0,
                "Full Transcript": f"Error: {str(e)}",
                "Quality Issues": str(e),
                "Additional Info": "Error en procesamiento"
            })


    df_results = pd.DataFrame(results)
    total_time = (datetime.now() - start_time).total_seconds()
    return df_results, total_time

def generate_statistics_markdown(df, total_time):
    """Genera estadísticas en formato markdown."""
    total_calls = len(df)
    if total_calls == 0:
        return "Sin datos para mostrar estadísticas"

    # Ensure columns exist before accessing
    flags_count_col = df.get('Flags Count', pd.Series(dtype=int))
    compliance_col = df.get('Compliance', pd.Series(dtype=str))
    vertical_col = df.get('Vertical', pd.Series(dtype=str))

    calls_with_flags = len(flags_count_col[flags_count_col > 0])
    compliance_rate = len(compliance_col[compliance_col == 'Cumple']) / total_calls * 100 if total_calls > 0 else 0
    vertical_counts = vertical_col.value_counts().to_dict()

    # Calculate time in minutes and seconds
    minutes = int(total_time // 60)
    seconds = int(total_time % 60)
    time_str = f"{minutes} min {seconds} seg" if minutes > 0 else f"{seconds} seg"


    stats = f"""
## 📊 Resumen de Análisis
- **Total de Llamadas**: {total_calls}
- **Llamadas con Marcas/Flags**: {calls_with_flags} ({calls_with_flags/total_calls*100:.1f}%)
- **Tasa de Cumplimiento**: {compliance_rate:.1f}%
- **Tiempo Total**: {time_str}

### Distribución por Vertical
"""
    for vertical, count in vertical_counts.items():
        stats += f"- {vertical}: {count} ({count/total_calls*100:.1f}%)\n"

    # Add flags count if 'Flags Found' column exists
    if 'Flags Found' in df.columns:
        all_flags = []
        # Iterate through the Series and handle potential NaN or non-string values
        for flags_str in df['Flags Found'].dropna():
            if isinstance(flags_str, str) and flags_str != 'Ninguna' and flags_str != 'Error':
                 all_flags.extend([f.strip() for f in flags_str.split(',')])

        if all_flags:
            flag_counts = pd.Series(all_flags).value_counts().head(10) # Show top 10 flags
            stats += "\n### Top Flags Encontrados\n"
            for flag, count in flag_counts.items():
                stats += f"- **{flag}**: {count} veces\n"

    # Add appointment count if 'Transcript Summary' column exists and contains "cita programada"
    if 'Transcript Summary' in df.columns:
        appointment_count = len(df[df['Transcript Summary'].str.contains('cita programada', na=False)])
        stats += f"\n- **Llamadas con Cita Programada**: {appointment_count}\n"


    return stats

def export_enhanced_excel(df):
    """Exporta los resultados a un archivo Excel con formato mejorado."""
    try:
        # Exporting to /content directory
        output_path = f'/content/call_analysis_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
        df.to_excel(output_path, index=False, engine='openpyxl')
        logger.info(f"Excel exportado a {output_path}")
        return output_path
    except Exception as e:
        logger.error(f"Error exportando Excel: {e}")
        return None

# Keep the EXCEL_COLUMN_MAPPING from the first cell
EXCEL_COLUMN_MAPPING = {
    'recording_link': [
        'Recording Link', 'Recording URL', 'recording_url', 'URL', 'url',
        'Audio URL', 'Link', 'recording_link', 'audio_link', 'call_recording'
    ],
    'caller_id': [
        'Caller ID', 'CallID', 'caller_id', 'ID', 'Phone', 'Caller',
        'phone_number', 'PhoneNumber', 'lead_phone', 'customer_phone', 'From' # Added 'From'
    ],
    'publisher': ['Publisher', 'publisher', 'Pub', 'Source', 'Traffic Source', 'Partner'],
    'buyer': ['Buyer', 'buyer', 'Client', 'Customer', 'Account'],
    'date': ['Date', 'date', 'Call Date', 'Timestamp', 'Created Date', 'call_date'],
    'time': ['Time', 'time', 'Call Time', 'Duration', 'Call Duration', 'Length'],
    'campaign': ['Campaign', 'campaign', 'Campaign Name', 'Campaign ID'],
    'lead_id': ['Lead ID', 'lead_id', 'LeadID', 'Lead', 'Lead Number'],
    'vertical': ['Vertical', 'vertical', 'Industry', 'Category', 'Product'],
    'status': ['Status', 'status', 'Call Status', 'Disposition']
}


def create_ui():
    excel_input = widgets.FileUpload(accept='.xlsx,.xls', multiple=False, description='Subir Excel')
    audio_input = widgets.FileUpload(accept='.mp3,.wav,.m4a', multiple=True, description='Subir Audios')
    url_input = widgets.Textarea(value='', placeholder='https://ejemplo.com/audio1.mp3, https://ejemplo.com/audio2.mp3', description='URLs:', layout={'width': '600px', 'height': '100px'})
    process_button = widgets.Button(description='Procesar Todo', button_style='primary')
    progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progreso:', bar_style='info', layout={'width': '600px'})
    status_label = widgets.Label(value='Listo para procesar')
    status_output = widgets.Output()
    results_output = widgets.Output()
    stats_output = widgets.Output()

    def process_all_inputs(b):
        with status_output:
            status_output.clear_output()
            results_output.clear_output()
            stats_output.clear_output()
            progress_bar.value = 0.0
            status_label.value = "⏳ Iniciando procesamiento..."

        files = []
        for filename, file_info in audio_input.value.items():
            if not filename.lower().endswith(('.mp3', '.wav', '.m4a')):
                with status_output:
                    print(f"❌ Archivo {filename} no soportado")
                continue
            with tempfile.NamedTemporaryFile(suffix=os.path.splitext(filename)[1], delete=False) as temp_file:
                temp_file.write(file_info['content'])
                files.append(type('obj', (object,), {'name': temp_file.name}))

        excel_file = None
        if excel_input.value:
            filename = list(excel_input.value.keys())[0]
            if not filename.lower().endswith(('.xlsx', '.xls')):
                with status_output:
                    status_output.clear_output()
                    status_label.value = f"❌ Archivo {filename} no es un Excel válido"
                return
            with tempfile.NamedTemporaryFile(suffix='.xlsx', delete=False) as temp_file:
                temp_file.write(excel_input.value[filename]['content'])
                excel_file = temp_file.name

        def progress_callback(progress, message):
            with status_output:
                status_output.clear_output()
                progress_bar.value = progress
                status_label.value = f"{message} ({progress*100:.1f}%)"

        try:
            df, total_time = process_mixed_inputs(files, url_input.value, excel_file, progress_callback)
            if len(df) == 0:
                with status_output:
                    status_output.clear_output()
                    progress_bar.value = 0.0
                    status_label.value = "❌ No se encontraron datos para procesar"
                # Clean up temp files
                for temp_file in files + ([excel_file] if excel_file else []):
                    if temp_file and os.path.exists(temp_file.name if hasattr(temp_file, 'name') else temp_file):
                        try:
                            os.remove(temp_file.name if hasattr(temp_file, 'name') else temp_file)
                        except Exception as e:
                            logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")
                # Reset widgets
                excel_input.value.clear()
                audio_input.value.clear()
                url_input.value = ''
                excel_input._counter = 0
                audio_input._counter = 0
                return

            # Ensure necessary columns for preview exist, add if not
            preview_cols_base = ["Caller ID", "Vertical", "Flags Found", "Transcript Summary", "Compliance"]
            preview_cols = [col for col in preview_cols_base if col in df.columns]

            preview_df = df[preview_cols].head(50)
            stats_md = generate_statistics_markdown(df, total_time)
            excel_path = export_enhanced_excel(df)

            with results_output:
                results_output.clear_output()
                display(HTML("<h3>Resultados (Primeras 50 Filas)</h3>"))
                display(preview_df)

            with stats_output:
                stats_output.clear_output()
                display(HTML("<h3>Estadísticas</h3>"))
                display(HTML(stats_md.replace('\n', '<br>')))

            with status_output:
                status_output.clear_output()
                progress_bar.value = 1.0
                if excel_path:
                    status_label.value = f"✅ Procesamiento completado: {len(df)} audios analizados"
                    display(HTML(f'<p>Reporte Excel guardado en: <b>{excel_path}</b></p>'))
                    display(HTML(f'<a href="{excel_path}" download>Descargar Excel</a>'))
                else:
                    status_label.value = f"⚠️ Procesados {len(df)} audios, pero error en exportar Excel"
                    display(HTML("<p>Hubo un error al exportar el archivo Excel. Verifique los logs para más detalles.</p>"))


            # Clean up temp files after processing
            for temp_file in files + ([excel_file] if excel_file else []):
                 if temp_file and os.path.exists(temp_file.name if hasattr(temp_file, 'name') else temp_file):
                    try:
                        os.remove(temp_file.name if hasattr(temp_file, 'name') else temp_file)
                    except Exception as e:
                        logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")

            # Reset widgets
            excel_input.value.clear()
            audio_input.value.clear()
            url_input.value = ''
            excel_input._counter = 0
            audio_input._counter = 0


        except Exception as e:
            with status_output:
                status_output.clear_output()
                progress_bar.value = 0.0
                status_label.value = f"❌ Error: {str(e)}"
            # Clean up temp files in case of error
            for temp_file in files + ([excel_file] if excel_file else []):
                if temp_file and os.path.exists(temp_file.name if hasattr(temp_file, 'name') else temp_file):
                    try:
                        os.remove(temp_file.name if hasattr(temp_file, 'name') else temp_file)
                    except Exception as e:
                        logger.warning(f"Error eliminando archivo temporal {temp_file}: {e}")
             # Reset widgets
            excel_input.value.clear()
            audio_input.value.clear()
            url_input.value = ''
            excel_input._counter = 0
            audio_input._counter = 0



    process_button.on_click(process_all_inputs)
    display(HTML("<h2>🎧 Análisis Avanzado de Llamadas</h2>"))
    display(HTML("""
        <p><b>Funcionalidades:</b></p>
        <ul>
            <li>📊 Análisis desde Excel con URLs de Grabación</li>
            <li>🎯 Detección de Flags (Ej: 'free roofing', 'government program')</li>
            <li>📅 Detección de Citas Programadas</li>
            <li>📈 Reportes detallados en Excel y Resumen en Notebook</li>
            <li>🌐 Soporte multilingüe</li>
        </ul>
        <p><b>Formato Excel:</b> Debe contener una columna con las URLs de grabación (Ej: 'Recording Link', 'Recording URL'). Columnas opcionales para metadatos: 'Publisher', 'Buyer', 'Caller ID', 'From'.</p>
    """))
    display(excel_input)
    display(audio_input)
    display(url_input)
    display(process_button)
    display(progress_bar)
    display(status_label)
    display(status_output)
    display(results_output)
    display(stats_output)

# Ensure EXCEL_COLUMN_MAPPING is defined before calling create_ui
EXCEL_COLUMN_MAPPING = {
    'recording_link': [
        'Recording Link', 'Recording URL', 'recording_url', 'URL', 'url',
        'Audio URL', 'Link', 'recording_link', 'audio_link', 'call_recording'
    ],
    'caller_id': [
        'Caller ID', 'CallID', 'caller_id', 'ID', 'Phone', 'Caller',
        'phone_number', 'PhoneNumber', 'lead_phone', 'customer_phone', 'From'
    ],
    'publisher': ['Publisher', 'publisher', 'Pub', 'Source', 'Traffic Source', 'Partner'],
    'buyer': ['Buyer', 'buyer', 'Client', 'Customer', 'Account'],
    'date': ['Date', 'date', 'Call Date', 'Timestamp', 'Created Date', 'call_date'],
    'time': ['Time', 'time', 'Call Time', 'Duration', 'Call Duration', 'Length'],
    'campaign': ['Campaign', 'campaign', 'Campaign Name', 'Campaign ID'],
    'lead_id': ['Lead ID', 'lead_id', 'LeadID', 'Lead', 'Lead Number'],
    'vertical': ['Vertical', 'vertical', 'Industry', 'Category', 'Product'],
    'status': ['Status', 'status', 'Call Status', 'Disposition']
}


create_ui()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

...

Label(value='Listo para procesar')

Output()

Output()

Output()

ERROR:__main__:Error procesando Excel /tmp/tmpj2f4c5a7.xlsx: El archivo Excel debe contener una columna de URL válida. Columnas buscadas: ['Recording Link', 'Recording URL', 'recording_url', 'URL', 'url', 'Audio URL', 'Link', 'recording_link', 'audio_link', 'call_recording']
